# Python in Comparison with Other Languages

Someone that is familiar with arrays in the C language could write Python code similar to the following example:


In [1]:
some_list = []
for index in range(len(some_list)):
    print(some_list[index])


An experienced Pythonic programmer would most probably write:


In [2]:
for item in some_list:
    print(item)

## Accessing super-classes

Consider a subclass of Python's dictionary type, which allows access to the stored keys through a case-insensitive key lookup. 
The following is an example of such an implementation:


In [3]:
!python _01_Accessing_super_classes/caseinsensitive.py

FOO: bar
foo: bar
biz: baz
BIZ: baz


Good practices when subclassing built-in types:
* Trying to directly subclass the built-in types 'dict' and 'list' can lead to subtle bugs.
* For this reason, use colections.UserDict and collections.UserList for subclassing the dict and list types, respectively.


## Multiple inheritance and Method Resolution Order

The MRO algorithm in python is based on the 'C3 linearization' algorithm. It's useful in 'diamond class hierarchy' scenarios, such as the following example:


In [4]:
!python _02_Multiple_iheritance_MRO/mro.py

("MyClass's MRO: \n"
 "(<class '__main__.MyClass'>, <class '__main__.Base1'>, <class "
 "'__main__.Base2'>, <class '__main__.CommonBase'>, <class 'object'>)")


## Class instance initialization

Best practices when initializing classes:
* Python classes do not require you to define attributes in the class body.
* Defining attributes in the class body is also **dangerous**: it can lead to very subtle bugs if one decides to assign as a class attribute a mutable type like list or dict.


To see the issues with mutable class attributes in action, consider the following class with mutable types as class attributes:

In [5]:
from _03_Class_instance_init.aggregator_shared import AggregatorShared


Let's create 2 instances of class 'AggregatorShared' and start adding elements to the 2 instances:

In [6]:
a1 = AggregatorShared()
a2 = AggregatorShared()
a1.aggregate("a1-1")
a1.aggregate("a1-2")
a2.aggregate("a2-1")


When checking the 'all_aggregated' attribute of both instances, they are the same!. It's like they're not storing their own values in 'all_aggregated', but sharing all values added in a single list:

In [7]:
a1.all_aggregated

['a1-1', 'a1-2', 'a2-1']

In [8]:
a2.all_aggregated

['a1-1', 'a1-2', 'a2-1']

When checking the 'last_aggregated' attribute of both instances, they are the last values added in each instance, as expected:

In [9]:
a1.last_aggregated


'a1-2'

In [10]:
a2.last_aggregated


'a2-1'

By inspecting the class attribute values, we see that all 'AggregatorShared' instances shared their state
through the mutable 'all_aggregated' attribute:

In [11]:
AggregatorShared.all_aggregated


['a1-1', 'a1-2', 'a2-1']

In [12]:
AggregatorShared.last_aggregated


To avoid the last issue, all attribute values that are supposed to be unique for every class instance should be initialized in the `__init__()` method only.

Now let's create 2 instances of class 'AggregatorIndependent' and start adding elements to the 2 instances:


In [13]:
from _03_Class_instance_init.aggregator_independent import AggregatorIndependent


a1 = AggregatorIndependent()
a2 = AggregatorIndependent()
a1.aggregate("a1-1")
a1.aggregate("a1-2")
a2.aggregate("a2-1")


Now all instance objects attributes have their own values in each attribute, as expected:

In [14]:
a1.all_aggregated


['a1-1', 'a1-2']

In [15]:
a2.all_aggregated

['a2-1']

In [16]:
a1.last_aggregated

'a1-2'

In [17]:
a2.last_aggregated

'a2-1'

## Attribute access patterns

Python lacks the notion of public, private, and protected class attributes found in other OOP languages:

* Private attributes restrict access to specific symbols from anyone outside of a specific class.

* Protected attributes restrict access to specific symbols from anyone outside of the inheritance tree.


The only similar thing Python has is 'name mangling': if an attribute is prefixed by __ (two underscores) within a class body, it's renamed by the interpreter:

In [18]:
class MyClass:
    def __init__(self):
        self.__secret_value = 1


instance_of = MyClass()


In [19]:
try:
    instance_of.__secret_value
except AttributeError as e:
    print(f'ERROR: {repr(e)}')

ERROR: AttributeError("'MyClass' object has no attribute '__secret_value'")


In [20]:
instance_of._MyClass__secret_value

1

In Python:

* Name mangling does not restrict attribute access, it only makes it less convenient.

* But it can help avoid naming collisions in the inheritance tree. Still, it's not recommended to use name mangling in base classes by default, to avoid any collisions in advance.


## Descriptors

The descriptor classes are based on 3 special methods that form the descriptor protocol:

* `__set__`(self, obj, value): This is called whenever the attribute is set. (a.k.a. setter)

* `__get__`(self, obj, owner=None): This is called whenever the attribute is read. (a.k.a. getter)

* `__delete__`(self, obj): This is called when del is invoked on the attribute.

* A descriptor that implements `__get__()` and `__set__()` is called a data descriptor.

* If it just implements `__get__()`, then it is called a non-data descriptor.


Consider the following example that shows how descriptors work: 

In [21]:
!python _04_Descriptors/reveal_access.py


Retrieving var "x"
10
Updating var "x"
Retrieving var "x"
20
5
Deleting var "x"


The preceding example shows that:

* The `__get__()` method is called whenever the instance attribute is retrieved. 

* The `__set__()` method is called whenever a value is assigned to the instance attribute. 

* The `__del__()` method is called whenever an instance attribute is deleted.

* Descriptors, in order to work, need to be defined as class attributes. 


## Lazily evaluated attributes

As an example usage of descriptors, consider a class 'WithSortedRandoms' where all instances have access to a shared list of sorted random values: 

* The length of the list can be arbitrarily long, so it makes sense to sort it once and reuse it for all instances. 

* So the list will be initialized only on first access. 

Here is an example usage of the 'WithSortedRandoms' class:


In [22]:
!python _05_lazily_evaluated_attributes/lazily_evaluated.py


initialized!
[0.016598060362082223, 0.10481644991803463, 0.22574921863577746, 0.5336030534000878, 0.9717477540254368]
cached!
[0.016598060362082223, 0.10481644991803463, 0.22574921863577746, 0.5336030534000878, 0.9717477540254368]


Also, a data descriptor can be implemented to be used as a decorator, as follows:


In [23]:
!python _05_lazily_evaluated_attributes/lazy_property.py


[[0.06911442085451336, 0.9375619584750133, 0.45047734675517925, 0.1159088773628445, 0.05134171065994331]]
[[0.06911442085451336, 0.9375619584750133, 0.45047734675517925, 0.1159088773628445, 0.05134171065994331]]


This is useful when:
* An object instance needs to be stored as a class attribute that's shared between its instances (to save resources).
* The object can't be initialized at import time because some global application state/context is needed.


## Properties


In other languages:

* It's common to have private methods by default and a getter an setter for every attribute.

* But with a large number of attributes, this means tons of getters and setter methods, which obscures the intent of the class.
 

In [24]:
class UserAccount:
    def __init__(self, username, password):
        self._username = username
        self._password = password

    def get_username(self):
        return self._username

    def set_username(self, username):
        self._username = username

    def get_password(self):
        return self._password

    def set_username(self, password):
        self._password = password


With python's properties:

* You can expose class attributes as public by default.

* Then convert one attribute to private and add getter and setter methods when necessary.


In [25]:
class UserAccount:
    def __init__(self, username, password):
        self.username = username
        self._password = password

    @property
    def password(self):
        return self._password
    
    @password.setter
    def password(self, value):
        self._password = value


## Dynamic polymorphism

Python's mechanism of polymorphism is called "duck typing": 

* This means that any object can be used within a given context as long as the object works and behaves as the context expects.

* "If it walks like a duck and it quacks like a duck, then it must be a duck".


Consider the following example:

In [26]:
def printfile(file):
    try:
        contents = file.read()
        print(file)
    finally:
        file.close()


This function won't raise any exception as long as:

* The file argument has a read() method.

* The result of file.read() is a valid argument to the print() function.

* The file argument has the close() method.


## Dunder methods (language protocols)

To see dunder methods in action, consider the following implementation of matrix operations (+, -, *):

In [27]:
!python _06_Dunder_methods/matrices.py


Matrix m1: 
[1, 2, 3]
[4, 1, 4]
[5, 7, 9]

Matrix m2: 
[1, 2, 3]
[1, 4, 3]
[1, 0, 5]

Sum of m1 and m2: 
[2, 4, 6]
[5, 5, 7]
[6, 7, 14]

Substraction of m1 and m2: 
[0, 0, 0]
[3, -3, 1]
[4, 7, 4]

Multiplication of m1 and m2: 
[6, 10, 24]
[9, 12, 35]
[21, 38, 81]



We can add support for multiplication between a matrix and a scalar as follows:

In [28]:
!python _06_Dunder_methods/matrices_with_scalars.py


Matrix m1: 
[1, 2, 3]
[4, 1, 4]
[5, 7, 9]

Matrix m2: 
[1, 2, 3]
[1, 4, 3]
[1, 0, 5]

Multiplication of m1 by 2: 
[2, 4, 6]
[8, 2, 8]
[10, 14, 18]

Multiplication of m2 by 3: 
[3, 6, 9]
[3, 12, 9]
[3, 0, 15]



## Single dispatch

In situations when an alternative to function overloading is required and the
number of alternative function implementations is really large, using multiple if
isinstance(...) clauses can quickly get out of hand. Good design practice dictates
writing small, single-purpose functions. One large function that branches over
several types to handle input arguments differently is rarely a good design.
The Python Standard Library provides a convenient alternative. The functools.
singledispatch() decorator allows you to register multiple implementations
of a function. Those implementations can take any number of arguments but
implementations will be dispatched depending on the type of the first argument.

In [29]:
!python _07_Single_dispatch/dispatch.py

dt: 2023-12-13T16:04:47.338100
complex: 100.0-30.0j
raw: January
<class 'object'> -> <function report at 0x7f83cff63d90>
<class 'datetime.datetime'> -> <function _ at 0x7f83cfcda170>
<class 'complex'> -> <function _ at 0x7f83cfcda200>
<class 'numbers.Real'> -> <function _ at 0x7f83cfcda3b0>


## Data classes

Consider a program that does some geometric computation with a 'Vector' class that holds info about 2D vectors. 

With a normal class, we have to implement all the methods related to math operations (addition, subtraction, equality) by hand, as follows:


In [30]:
def print_vec_operations(v1, v2):
    add = v1 + v2
    sub = v1 - v2
    uneq = (v1 == v2)
    eq = (v2 == v2)
    print(f"Adding vectors {v1} + {v2} = {add}")
    print(f"Substracting vectors {v1} - {v2} = {sub}")
    print(f"Comparing vectors {v1} and {v2} = {'equal' if uneq else 'unequal'}")
    print(f"Comparing vectors {v2} + {v2} = {'equal' if eq else 'unequal'}")


In [31]:
from _08_Dataclasses.vector import Vector


v1 = Vector(5, 3)
v2 = Vector(1, 2)

print_vec_operations(v1, v2)


Adding vectors <Vector: x=5, y=3> + <Vector: x=1, y=2> = <Vector: x=6, y=5>
Substracting vectors <Vector: x=5, y=3> - <Vector: x=1, y=2> = <Vector: x=4, y=1>
Comparing vectors <Vector: x=5, y=3> and <Vector: x=1, y=2> = unequal
Comparing vectors <Vector: x=1, y=2> + <Vector: x=1, y=2> = equal


But with python dataclasses, we can make our code shorter, since the `dataclasses` module automatically implements the `__init__()`, `__repr__()`, and `__eq__()` methods:
 

In [32]:
from _08_Dataclasses.vector_dataclasses import Vector as VectorDataClass


v1_dc = VectorDataClass(5, 3)
v2_dc = VectorDataClass(1, 2)

print_vec_operations(v1_dc, v2_dc)


Adding vectors Vector(x=5, y=3) + Vector(x=1, y=2) = Vector(x=6, y=5)
Substracting vectors Vector(x=5, y=3) - Vector(x=1, y=2) = Vector(x=4, y=1)
Comparing vectors Vector(x=5, y=3) and Vector(x=1, y=2) = unequal
Comparing vectors Vector(x=1, y=2) + Vector(x=1, y=2) = equal


Another useful feature is to make dataclasses immutable, as follows:

In [33]:
from _08_Dataclasses.vector_dataclasses import FrozenVector


v1_fr = FrozenVector(5, 3)
print(f"You can access attributes of class {v1_fr}. For example, its attribute 'x' is {v1_fr.x}")
print(f"But tou can't modify attributes of class {v1_fr}:")
try:
    v1_fr.x = 8
except Exception as e:
    print(f"\tERROR: {repr(e)}")

You can access attributes of class FrozenVector(x=5, y=3). For example, its attribute 'x' is 5
But tou can't modify attributes of class FrozenVector(x=5, y=3):
	ERROR: FrozenInstanceError("cannot assign to field 'x'")


Since assigning default values to class attributes in the class body is a bad idea, the dataclass module offers an alternative through the field() constructor, as follows:


In [34]:
from dataclasses import dataclass, field


@dataclass
class DataClassWithDefaults:
    immutable: str = field(default="this is static default value")
    mutable: list = field(default_factory=list)


In [35]:
DataClassWithDefaults()


DataClassWithDefaults(immutable='this is static default value', mutable=[])

In [36]:
DataClassWithDefaults("This is immutable")


DataClassWithDefaults(immutable='This is immutable', mutable=[])

In [37]:
DataClassWithDefaults(None, ["this", "is", "list"])


DataClassWithDefaults(immutable=None, mutable=['this', 'is', 'list'])

## Functional programming


### Some basic terminology of functional programming:

Side effects: 

* A function has side effects if it modifies the state outside of its local environment. 

* Examples of side effects: modification of a global variable or an attribute of an object that is available outside of the method scope,  saving data to some external service. 

Referential transparency: 

* A function is referentially transparent if it can be replaced with the value that corresponds to its output without changing the behavior of the program. 

* A referentially transparent function lacks side effects, but a function that lacks side effects may not be referentially transparent. 

* For instance, Python's pow(x, y) function lacks side effects and it can be replaced with the value of `x^y`, so it's referentially transparent; but the datetime.now() method doesn't have any observable side effects but will return a different value every time it's called, so it's referentially opaque. 

Pure functions: 

* A function that does not have any side effects and is referentially transparent.

First-class functions: 

* They are functions that can be treated as any other value or entity. 

* For example, they can be passed as arguments to other functions, returned as function return values, and assigned to variables. 


Python has functional features, such as:

* Lambda functions and first-class functions

* map(), filter(), and reduce() functions

* Partial objects and functions

* Generators and generator expression


## Lambda functions

Lambda functions are functions without name, commonly used once. Instead of writing this:

In [38]:
import math


def circle_area(radius):
    return math.pi * radius ** 2

circle_area(42)

5541.769440932395

You write this:

In [39]:
lambda radius: math.pi * radius ** 2

<function __main__.<lambda>(radius)>

In theory, you can assign lambda functions to variables, but it's not commonly used:

In [40]:
circle_area = lambda radius: math.pi * radius ** 2
circle_area(42)

5541.769440932395

The most common use is to use them as args to higher-order functions. For example, you can sort a list of objects by a custom attribute, as follows:

In [41]:
from dataclasses import dataclass


@dataclass
class Person:
    name: str
    age: int


people = [Person('Donald', 77), Person('Joe', 81), Person('Hillary', 76)]
sorted(people, key=lambda person: person.age)



[Person(name='Hillary', age=76),
 Person(name='Donald', age=77),
 Person(name='Joe', age=81)]

## The map(), filter(), and reduce() functions

In [42]:
import functools as ft
import itertools as it
import operator as op

The most common higher-order functions are map, filter and reduce. They are used as follows:

map() applies a function to every item of an iterable:

In [43]:
# The function map itself returns an iterator, not a sequence:

print(f"Map iterator object: {map(lambda x: x**2, range(10))}")

# To evaluate the iterator, pass the iterator object to a sequence:

print(f"Map object to list: {list(map(lambda x: x**2, range(10)))}")

# The map function can be applied to more than one sequence. It will stop at the length of the shortest one:

mapped = list(map(print, range(5), range(4), range(5)))

print(f"Map applied to 3 lists : {mapped}")





Map iterator object: <map object at 0x7f209c345c60>
Map object to list: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0 0 0
1 1 1
2 2 2
3 3 3
Map applied to 3 lists : [None, None, None, None]


filter() allows us to filter out input values that don't meet the predicate defined by the func argument:



In [44]:
evens = filter(lambda number: number % 2 == 0, range(10))
odds = filter(lambda number: number % 2 == 1, range(10))
print(f"Even numbers in range from 0 to 9 are: {list(evens)}")
print(f"Odd numbers in range from 0 to 9 are: {list(odds)}")

animals = ["giraffe", "snake", "lion", "squirrel"]
animals_s = filter(lambda animal: animal.startswith('s'), animals)
print(f"Animals that start with letter 's' are: {list(animals_s)}")


Even numbers in range from 0 to 9 are: [0, 2, 4, 6, 8]
Odd numbers in range from 0 to 9 are: [1, 3, 5, 7, 9]
Animals that start with letter 's' are: ['snake', 'squirrel']


reduce() reduce (as the name suggests) an iterable to a single value. It cumulatively performs an operation specified
by a functiona over all iterable items:

In [45]:
print(
    "Sum of the first 100 numbers using a lambda function: " + 
    f"{ft.reduce(lambda x, y: x + y, range(100))}"
)

print(
    "Sum of the first 100 numbers using operators from 'operator' module: " + 
    f"{ft.reduce(op.add, range(100))}"
)

list_ = [1, 3, 5, 6, 2] 
print(
    f"Finding the maximum element of list {list_}: " + 
    f"{ft.reduce(lambda x, y: x if x > y else y, list_)}"
)



Sum of the first 100 numbers using a lambda function: 4950
Sum of the first 100 numbers using operators from 'operator' module: 4950
Finding the maximum element of list [1, 3, 5, 6, 2]: 6


Both map() and filter() can work on infinite sequences and they return an iterator, so we can consume consecutive elements of the iterator using the next() function:


In [46]:
sequence = map(lambda x: x**2, it.count())
sequence

In [47]:
next(sequence)


0

In [48]:
next(sequence)

1

In [49]:
next(sequence)


4

In [50]:
next(sequence)


9

## Partial objects and functions

Partial functions are functions with some of its arguments 'fixed' to a value. For example, we can use `partial` to 'fix' the `base` argument of the `pow` function to 2, as follows:

In [51]:
powers_of_2 = ft.partial(pow, 2)


In [52]:
powers_of_2(2)


4

In [53]:
powers_of_2(5)


32

In [54]:
powers_of_2(10)


1024

## Generators

Generators allows us to write efficient code for functions that return sequences. Using the `yield` statement, we can pause a function and return an intermediate result. The function saves its execution context and can be resumed later.

For example, a generator expression to compute the Fibonacci sequence can be like the following:


In [55]:

def fibonacci():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a + b


We can retrieve new values from generators as if they were iterators with the `next()` function:

In [56]:
fib = fibonacci()
for idx in range(10):
    print(f'Item {idx + 1} in the Fibonacci sequence: {next(fib)}')


Item 1 in the Fibonacci sequence: 1
Item 2 in the Fibonacci sequence: 1
Item 3 in the Fibonacci sequence: 2
Item 4 in the Fibonacci sequence: 3
Item 5 in the Fibonacci sequence: 5
Item 6 in the Fibonacci sequence: 8
Item 7 in the Fibonacci sequence: 13
Item 8 in the Fibonacci sequence: 21
Item 9 in the Fibonacci sequence: 34
Item 10 in the Fibonacci sequence: 55


Also, we can retrieve generator values in for loops:

In [57]:
for idx, item in enumerate(fibonacci()):
    print(f'Item {idx + 1} in the Fibonacci sequence: {item}')
    if idx >= 9:
        break


Item 1 in the Fibonacci sequence: 1
Item 2 in the Fibonacci sequence: 1
Item 3 in the Fibonacci sequence: 2
Item 4 in the Fibonacci sequence: 3
Item 5 in the Fibonacci sequence: 5
Item 6 in the Fibonacci sequence: 8
Item 7 in the Fibonacci sequence: 13
Item 8 in the Fibonacci sequence: 21
Item 9 in the Fibonacci sequence: 34
Item 10 in the Fibonacci sequence: 55


## Decorators

In [58]:
from functools import wraps


Decorators are sytax sugar for higher-order functions. The function skeleton of a decorator is as follows:

In [59]:
def some_decorator(f):
    @wraps(f) # @wraps decorator copies decorated function metadata (name, type annotations)
    def decorated_function(*args, **kwargs):
        ## Do something here
        return f(*args, **kwargs)
    return decorated_function

To use decorators, use `@` before the decorator name, as follows:

In [60]:
@some_decorator
def decorated_function():
    pass

This is equifalent to the following function call:

In [61]:
decorated_function = some_decorator(decorated_function)


## Enumerations

In [62]:
from enum import Enum, Flag, auto


Enumerated types or Enums are types that have a finite number of named values. They are useful for encoding a closed set of values for variables or function arguments.

A simple Enum to encode the days of the week would be as follows:


In [63]:
class Weekday(Enum):
    MONDAY = 0
    TUESDAY = 1
    WEDNESDAY = 2
    THURSDAY = 3
    FRIDAY = 4
    SATURDAY = 5
    SUNDAY = 6



If the member values are not important, we can use the auto() type, which will be replaced with automatically generated values:


In [64]:
class Weekday(Enum):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()

Enums are useful when some variable can take only a finite number of choices. For example, we can use them to define the status of objects, as follows:


In [65]:
class OrderStatus(Enum):
    PENDING = auto()
    PROCESSING = auto()
    PROCESSED = auto()


class Order:
    def __init__(self):
        self.status = OrderStatus.PENDING

    def process(self):
        if self.status == OrderStatus.PROCESSED:
            raise ValueError(
                "Can't process order that has been already processed"
            )
        # Some code here...
        self.status = OrderStatus.PROCESSING
        # More code here...
        self.status = OrderStatus.PROCESSED
